# 1) Setting up Neural Network for Training
### batch_size
- `batch_size`:  how many images that we want to feed into the network at once during training.
- **Typical batch_size is between 32 and 128 images**, but can expirement freely.

If we set the number too low, training will take a long time and might not finish.  If we set the number too high, we'll run out of memory on our computer.

### epoch
- we need to decide how many times we want to go through our training data set during the process.  One full pass through the entire training dataset is called an `epoch`.  
- The more epoches we set, the more chance the neural network has to learn; but the longer training process will take. eventually you will hit the point where doing additional training doesn't help anymore. so finding the right number take expirmentation.
- The larger the data set, the less training passes you'll do on it. For extremelly large dataset with millions of images you might only do five passes, etc.

### shuffle
- we need to make sure Keras randomizes the order of training data. It's very important that neural network sees the training data baches in random order, so that the order of training data doesn't influcence the training. 


In [3]:
import tensorflow as tf
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout, Conv2D, MaxPool2D
from pathlib import Path

# load data
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

# Normalize data
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train = x_train / 255
x_test = x_test / 255

# Categorical encoding for labels
y_train = tf.keras.utils.to_categorical(y_train, 10)
y_test = tf.keras.utils.to_categorical(y_test, 10)

In [4]:
# Create model and Add layers
model = Sequential()

model.add(Conv2D(32, (3,3), padding='same', activation='relu', input_shape=(32, 32, 3))) # we want to put some padding
model.add(Conv2D(32, (3,3), activation='relu'))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3,3), padding='same', activation='relu'))
model.add(Conv2D(64, (3,3), activation='relu'))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Flatten())

model.add(Dense(512, activation='relu', input_shape=(32, 32, 3)))
model.add(Dropout(0.50))

model.add(Dense(10, activation='softmax')) # multiclass classification

# Compile the model
model.compile(
    loss='categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)

# Summary of model
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 32, 32, 32)        896       
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 30, 30, 32)        9248      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 15, 15, 32)        0         
_________________________________________________________________
dropout_3 (Dropout)          (None, 15, 15, 32)        0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 15, 15, 64)        18496     
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 13, 13, 64)        36928     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 6, 6, 64)         

In [8]:
# Training the model
model.fit(
    x_train,
    y_train,
    batch_size=64,
    epochs=30,
    validation_data=(x_test, y_test),
    shuffle=True
)

Epoch 1/30
782/782 [==============================] - 4s 5ms/step - loss: 0.4050 - accuracy: 0.8572 - val_loss: 0.6606 - val_accuracy: 0.7920
Epoch 2/30
782/782 [==============================] - 4s 5ms/step - loss: 0.3909 - accuracy: 0.8622 - val_loss: 0.6531 - val_accuracy: 0.7953
Epoch 3/30
782/782 [==============================] - 4s 5ms/step - loss: 0.3866 - accuracy: 0.8605 - val_loss: 0.6591 - val_accuracy: 0.7929
Epoch 4/30
782/782 [==============================] - 4s 5ms/step - loss: 0.3848 - accuracy: 0.8622 - val_loss: 0.6652 - val_accuracy: 0.7904
Epoch 5/30
782/782 [==============================] - 4s 6ms/step - loss: 0.3829 - accuracy: 0.8655 - val_loss: 0.6706 - val_accuracy: 0.7888
Epoch 6/30
782/782 [==============================] - 4s 5ms/step - loss: 0.3748 - accuracy: 0.8674 - val_loss: 0.6781 - val_accuracy: 0.7901
Epoch 7/30
782/782 [==============================] - 4s 5ms/step - loss: 0.3693 - accuracy: 0.8697 - val_loss: 0.6619 - val_accuracy: 0.7935
Epoch 

--------


# 2) Training Neural Network and Saving weights

After training completes, we want to save the trained neural network to a file. so we can use it to recognize objects and images in other programs.

## 2.1) Saving the neural network
- First we want to save the structure of the neural network itself. That includes which layers get created and the order they are hooked together.

## 2.2) Saving the neural network's trained weights
- Second we want to save weights of trained neural network. As neural network is trained, the weights of each nodes are adjusted to control how much the signals flow through the network. So by saving the weights, we are acutally saving how neural network actually works.
- HDF5 format is designed for saving and loading large binary files efficiently. by convention, we use extension `.h5` to indicate the format of the file.

In [10]:
# Save neural  network structure
model_structure = model.to_json()
f = Path('model_structure.json')
f.write_text(model_structure)

# Save neural network's trained weights
model.save_weights("model_weights.h5")

--------

# 3) Making Predictions with trained neural network
- Keras expects images to be provided in batches (even if we want to predict one image)
- there is one trick we can use if we want to predict only one image `numpy.expand_dims()`. We need to provide `axis=0` stating that new axis will be first dimension.

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
from tensorflow.keras.models import model_from_json
from tensorflow.keras.preprocessing import image

from pathlib import Path
import numpy as np

In [5]:
# These are the CIFAR10 class labels from the training data (in order from 0 to 9)
class_labels = [
    "Plane",
    "Car",
    "Bird",
    "Cat",
    "Deer",
    "Dog",
    "Frog",
    "Horse",
    "Boat",
    "Truck"
]

In [9]:
# Load the json file that contains the model's structure
file_path = '/content/drive/MyDrive/Deeplearning_image_recognition/'
f = Path(file_path + 'model_structure.json')
model_structure = f.read_text()

# Recreate the Keras model object from the json data
model = model_from_json(model_structure)

# Re-load the model's trained weights
model.load_weights(file_path + 'model_weights.h5')

#--------- Testing Image ------------- #
# Load an image file to test, resizing it to 32x32 pixels (as required by this model)
img = image.load_img(file_path + 'frog.png', target_size=(32, 32))

# Convert the image to a 3D numpy array, with normalized data
image_to_test = image.img_to_array(img) / 255

# Add a fourth dimension to the image (since Keras expects a list of images, not a single image)
list_of_images = np.expand_dims(image_to_test, axis=0)

# Make a prediction using the model
results = model.predict(list_of_images)

# Since we are only testing one image, we only need to check the first result
single_result = results[0]

# We will get a likelihood score for all 10 possible classes. Find out which class had the highest score.
most_likely_class_index = int(np.argmax(single_result))
class_likelihood = single_result[most_likely_class_index]

# Get the name of the most likely class
class_label = class_labels[most_likely_class_index]

# Print the result
print("This is image is a {} - Likelihood: {:2f}".format(class_label, class_likelihood))

This is image is a Frog - Likelihood: 0.978655
